In [ ]:
import nibabel as nib
import os
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import scipy.ndimage

In [ ]:
import itk
from itkwidgets import view
import itkwidgets

In [ ]:
#load lesion
file_name = "../data/origin data/"
datalist = os.listdir(file_name)


for i in range(len(datalist)):
    DWI_name = file_name+datalist[i]+'/lesion2MNI.nii.gz'
    

    DWI_nii = nib.load(DWI_name)
    DWI_img = np.asarray(DWI_nii.dataobj)
    print(datalist[i],"has",DWI_img.sum(),"lesion voxel.")

In [ ]:
#load aal3 brain atlas
label_name = '../data/MNI/AAL3_MNI.nii.gz'
label_nii = nib.load(label_name)
label_img = np.asarray(label_nii.dataobj)
print(label_img.shape)
print(label_img.max())
count = 0
for i in range(1,int(label_img.max())+1):
    modified_img = label_img.copy()
    modified_img[modified_img!=i] = 0
    modified_img[modified_img==i] = 1
    target = int(sum(sum(sum(modified_img))))
    count += target
    print(i,"has",target,"voxel")
print("totally",count,"voxel not equal to 0.")    

## show image

In [ ]:
#load MNI image
MNI_name = '../data/origin data/is0175/diff2MNI.nii.gz'
MNI_nii = nib.load(MNI_name)
MNI_img = np.asarray(MNI_nii.dataobj,dtype=np.int16)
print(MNI_img.shape)
print(MNI_img.sum())

In [ ]:
#check target brain atlas location

label_name = '../data/MNI/AAL3_MNI.nii.gz'
label_nii = nib.load(label_name)
label_img = np.asarray(label_nii.dataobj)
target = 8
label_img[label_img!=target] = 0
label_img[label_img==target] = 1
print(label_img.shape)
print(label_img.sum())

In [ ]:
#generate a target-only brain region mask in rank and save in nii.gz format
label_name = '../data/MNI/AAL3_MNI.nii.gz'
label_nii = nib.load(label_name)
label_img = np.asarray(label_nii.dataobj)
label_affine = label_nii.affine

label_img_2 = label_img.copy()
mRSimp3 = [8,85,69]
used = []
rank=1
for i in range(len(mRSimp3)):
    if mRSimp3[i] not in used:
        label_img_2[label_img==mRSimp3[i]] = rank
        used.append(mRSimp3[i])
        rank+=1
for i in range(int(label_img.max())):
    if i not in mRSimp3:
        label_img_2[label_img==i] = 0
        
new_img = nib.Nifti1Image(label_nii.get_fdata(), label_nii.affine, label_nii.header)
output_file = 'mRSimp_RFE_top3.nii.gz'
nib.save(new_img, output_file)

In [ ]:
image_view_from_array = itk.image_view_from_array((MNI_img))

print(image_view_from_array.shape)

viewer = view(image_view_from_array,label_image_weights=(1,1),ui_collapsed=True)
viewer

In [ ]:
label_view_from_array = itk.image_view_from_array((label_img))

print(label_view_from_array.shape)

viewer = view(label_view_from_array,label_image_weights=(1,1),ui_collapsed=True)
viewer

In [ ]:
#overlay
data_spacing = (1,1,1)

image_from_array = itk.image_from_array(MNI_img)
label_from_array = itk.image_from_array(label_img_2)

print(image_from_array.shape)
print(label_from_array.shape)

#image_from_array.SetSpacing(data_spacing)
#label_from_array.SetSpacing(data_spacing)

viewer = view(image_from_array, label_image=label_from_array,label_image_weights=(1,1),cmp=itkwidgets.cm.grayscale,ui_collapsed=True)
viewer